In [22]:
import numpy as np

njobs=10
nevents=4595295
startEvts=np.array([0])
stopEvts=np.array([nevents])
startEvts=np.fromfunction(lambda i: i*(nevents/njobs), (njobs,), dtype=int)
stopEvts=np.insert(stopEvts,0,np.fromfunction(lambda i: (i+1)*(nevents/njobs)-1,(njobs-1,), dtype=int))
StEvs=np.arange(njobs)

print startEvts
print stopEvts

[      0  459529  919058 1378587 1838116 2297645 2757174 3216703 3676232
 4135761]
[ 459528  919057 1378586 1838115 2297644 2757173 3216702 3676231 4135760
 4595295]


In [23]:
workDir = "/mnt/t3nfs01/data01/shome/threiten/QReg/test"
Y="probeR9"
quantiles = [.11,.22,.33,.44,.55,.66,.77,.88,.99]


#arg_list_mc=map(lambda q: ("/mnt/t3nfs01/data01/shome/threiten/QReg/dataMC-1/MTR/quantileRegression",
#                            "mc",
#                          (workDir + "/df_mc_All.h5",startEvt,stopEvt)),
#                           quantiles)

arg_list=map(lambda istEv: ("/mnt/t3nfs01/data01/shome/threiten/QReg/dataMC-1/MTR/quantileRegression",
                             "mc",
                            (workDir + "/df_mc_All.h5",startEvts[istEv],stopEvts[istEv]),
                            (workDir + "/data_weights","probeR9",quantiles),
                            (workDir + "/mc_weights","probeR9",quantiles)),
                             StEvs)

print arg_list


[('/mnt/t3nfs01/data01/shome/threiten/QReg/dataMC-1/MTR/quantileRegression', 'mc', ('/mnt/t3nfs01/data01/shome/threiten/QReg/test/df_mc_All.h5', 0, 459528), ('/mnt/t3nfs01/data01/shome/threiten/QReg/test/data_weights', 'probeR9', [0.11, 0.22, 0.33, 0.44, 0.55, 0.66, 0.77, 0.88, 0.99]), ('/mnt/t3nfs01/data01/shome/threiten/QReg/test/mc_weights', 'probeR9', [0.11, 0.22, 0.33, 0.44, 0.55, 0.66, 0.77, 0.88, 0.99])), ('/mnt/t3nfs01/data01/shome/threiten/QReg/dataMC-1/MTR/quantileRegression', 'mc', ('/mnt/t3nfs01/data01/shome/threiten/QReg/test/df_mc_All.h5', 459529, 919057), ('/mnt/t3nfs01/data01/shome/threiten/QReg/test/data_weights', 'probeR9', [0.11, 0.22, 0.33, 0.44, 0.55, 0.66, 0.77, 0.88, 0.99]), ('/mnt/t3nfs01/data01/shome/threiten/QReg/test/mc_weights', 'probeR9', [0.11, 0.22, 0.33, 0.44, 0.55, 0.66, 0.77, 0.88, 0.99])), ('/mnt/t3nfs01/data01/shome/threiten/QReg/dataMC-1/MTR/quantileRegression', 'mc', ('/mnt/t3nfs01/data01/shome/threiten/QReg/test/df_mc_All.h5', 919058, 1378586), ('

In [32]:
import ipyparallel as ipp
rc = ipp.Client(profile='short')
dv=rc[:]
dv.activate()
rc.ids
dv.scatter('args',arg_list)
dv.targets

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [33]:
%%px --noblock

import numpy as np

lib, tpe, args_df, args_dataw, args_mcw = args[0]
from import_file import import_file
QReg = import_file(lib)

qReg_mc = QReg.quantileRegression(tpe)
qReg_mc.loadDFh5(*args_df)
qReg_mc.loadDataWeights(*args_dataw)
qReg_mc.loadMcWeights(*args_mcw)

x = ['probePt', 'probeEta', 'probePhi', 'rho']
y = 'probeR9'

#print "Get corrections for ", y, " with quantiles ", quantiles

y_tmp = []

# quantile regressions features                                                                    
X    = qReg_mc.df.loc[:,x].values
# target e.g. y = "R9"                                                                             
Y    = qReg_mc.df[y]
print "Features: X = ", x, " target y = ", y
# print X, Y                                                                                       

if y == 'PhoIso03' or y == 'ChIso03' or y == 'ChIso03worst':
    Y = Y - 0.1*qReg_mc.df['rho']
    
Y = Y.values.reshape(-1,1)
Z = np.hstack([X,Y])

Y_corr = QReg.Corrector(qReg_mc.mcclf, qReg_mc.dataclf, Z[:,:-1], Z[:,-1])()
qReg_mc.df[y+"_corr"] = Y_corr

print Y_corr

<AsyncResult: execute>

In [34]:
%pxresult

[stdout:0] 
Welcome to ROOTaaS 6.06/05
Loading dataframe from :  /mnt/t3nfs01/data01/shome/threiten/QReg/test/df_mc_All.h5
Selecting events   [ 0 ,  459528 ]  out of  4595295
number of events: 459528
Features: X =  ['probePt', 'probeEta', 'probePhi', 'rho']  target y =  probeR9
[ 0.82588571  0.86952533  0.95608161 ...,  0.94233006  0.8956519
  0.96886695]
[stdout:1] 
Welcome to ROOTaaS 6.06/05
Loading dataframe from :  /mnt/t3nfs01/data01/shome/threiten/QReg/test/df_mc_All.h5
Selecting events   [ 459529 ,  919057 ]  out of  4595295
number of events: 459528
Features: X =  ['probePt', 'probeEta', 'probePhi', 'rho']  target y =  probeR9
[ 0.93850304  0.94611547  0.76837734 ...,  0.79560124  0.67712743
  0.95411176]
[stdout:2] 
Welcome to ROOTaaS 6.06/05
Loading dataframe from :  /mnt/t3nfs01/data01/shome/threiten/QReg/test/df_mc_All.h5
Selecting events   [ 919058 ,  1378586 ]  out of  4595295
number of events: 459528
Features: X =  ['probePt', 'probeEta', 'probePhi', 'rho']  target y =  p

[output:0]

[output:1]

[output:2]

[output:3]

...

[output:5]

[output:6]

[output:7]

[output:8]

[output:9]

In [35]:
df_all = dv.gather('qReg_mc')

In [36]:
df_dict= df_all.get_dict()

CompositeError: one or more exceptions from call to method: gather
[Engine Exception]EngineError: Engine 0 died while running task '4f831774-0dde-4594-a886-103a22f2494a'
[Engine Exception]EngineError: Engine 1 died while running task '9f1d4b40-022d-4122-9e32-2a9c95e157ca'
[Engine Exception]EngineError: Engine 2 died while running task '12fcdb8c-b7e1-434c-983a-a9cbace7fc43'
[Engine Exception]EngineError: Engine 3 died while running task 'e6aac149-70d1-477f-97fc-24270c32ab42'
.... 6 more exceptions ...